In [1]:
# Install the needed dependencies
import sys
#!{sys.executable} -m pip install imblearn

In [2]:
# Import the modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.utils.validation import check_array
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from pymongo import MongoClient
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# assign the uk_food database to a variable name
db = mongo['credit_data']

In [5]:
# review the collections in our database
print(db.list_collection_names())

['ETL_credit_data']


In [6]:
# assign the collection to a variable
credits = db['ETL_credit_data']

In [7]:
# Create lists to store the data
Current_Loan_Amount = []
Credit_Score = []
Annual_Income = []
Monthly_Income = []
Monthly_Debt = []
Debt_Income_Ratio = []
Credit_History = []
Months_Delinquent = []
Open_Accounts = []
Credit_Problems = []
Credit_Balance = []
Total_Credit = []
Credit_Usage_Ratio = []
Bankruptcies = []
Loan_Purpose = []

# for loop to extract the data from 'credits["ETL_credit_data"]'
for credit in credits["ETL_credit_data"].find():
    Current_Loan_Amount.append(credit['Current Loan Amount'])
    Credit_Score.append(credit['Credit Score'])
    Annual_Income.append(credit['Annual Income'])
    Monthly_Income.append(credit['Monthly Income'])
    Monthly_Debt.append(credit['Monthly Debt'])
    Debt_Income_Ratio.append(credit['Debt / Income Ratio'])
    Credit_History.append(credit['Credit History'])
    Months_Delinquent.append(credit['Months Delinquent'])
    Open_Accounts.append(credit['Open Accounts'])
    Credit_Problems.append(credit['Credit Problems'])
    Credit_Balance.append(credit['Credit Balance'])
    Total_Credit.append(credit['Total Credit'])
    Credit_Usage_Ratio.append(credit['Credit Usage Ratio'])
    Bankruptcies.append(credit['Bankruptcies'])
    Loan_Purpose.append(credit['Loan Purpose'])
    
# Create the DataFrame
ETL_credit_data = pd.DataFrame({
    "Current Loan Amount": Current_Loan_Amount,
    "Credit Score": Credit_Score,
    "Annual Income": Annual_Income,
    "Monthly Income": Monthly_Income,
    "Monthly Debt": Monthly_Debt,
    "Debt / Income Ratio": Debt_Income_Ratio,
    "Credit History(Years)": Credit_History,
    "Months Delinquent": Months_Delinquent,
    "Open Accounts": Open_Accounts,
    "Credit Problems": Credit_Problems,
    "Credit Balance": Credit_Balance,
    "Total Credit": Total_Credit,
    "Credit Usage Ratio": Credit_Usage_Ratio,
    "Bankruptcies": Bankruptcies,
    "Loan Purpose": Loan_Purpose
})

# Display Dataframe
ETL_credit_data

,Current Loan Amount,Credit Score,Annual Income,Monthly Income,Monthly Debt,Debt / Income Ratio,Credit History(Years),Months Delinquent,Open Accounts,Credit Problems,Credit Balance,Total Credit,Credit Usage Ratio,Bankruptcies,Loan Purpose
